# DBATools Log Shipping Demo
## Configure Log Shipping

In [107]:
Remove-Item -Path "C:\LogShipping\LSTest\*";
Remove-Item -Path "\\localhost\LogShippingDest\LSTest\*";
Remove-Item -Path "C:\LogShippingDest\LSTest\*";

$params = @{
 SourceSqlInstance = 'localhost'
 DestinationSqlInstance = 'localhost\tony'
 Database = 'LSTest'
 SharedPath= '\\localhost\LogShipping'
 LocalPath= 'C:\logshipping'
 BackupScheduleFrequencyType = 'daily'
 BackupScheduleFrequencyInterval = 1
 BackupScheduleDisabled = $true
 CompressBackup = $true
 CopyScheduleFrequencyType = 'daily'
 CopyScheduleFrequencyInterval = 1
 CopyScheduleDisabled = $true
 GenerateFullBackup = $true
 RestoreScheduleFrequencyType = 'daily'
 RestoreScheduleFrequencyInterval = 1
 RestoreScheduleDisable = $true
 SecondaryDatabaseSuffix = 'DR'
 CopyDestinationFolder = '\\localhost\LogShippingDest'
 Force = $true
 Standby = $true
 DisconnectUsers = $true
 }

Invoke-DbaDbLogShipping @params;




PrimaryInstance   : FD-PF33VX6J
SecondaryInstance : FD-PF33VX6J\TONY
PrimaryDatabase   : LSTest
SecondaryDatabase : LSTestDR
Result            : Success
Comment           : 





## Select from Primary and Secondary Tables

In [108]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;


Column1          Col1 Col2 Col3    
-------          ---- ---- ----    
FD-PF33VX6J         1 Log  Shipping
FD-PF33VX6J\TONY    1 Log  Shipping




## Insert Into Primary Table

In [109]:
$insert = @"
INSERT INTO TestTable
(Col2
,Col3)
VALUES
('Is'
,'Resilient')
"@
$primary = "localhost";
$primarydb = "LSTest";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $insert;

. {


>> $insert = @"
>> INSERT INTO TestTable
>> (Col2
>> ,Col3)
>> VALUES
>> ('Is'
>> ,'Resilient')
>> "@
>> $primary = "localhost";
>> $primarydb = "LSTest";
>> 
>> Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $insert;
>> }
>> 


## Select from Primary and Secondary Tables

In [110]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;



Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J         1 Log  Shipping 
FD-PF33VX6J         2 Is   Resilient
FD-PF33VX6J\TONY    1 Log  Shipping 




## Execute Log Shipping Jobs

In [111]:
Start-DbaAgentJob -SqlInstance "localhost" -Job LSBackup_LSTest;
Start-Sleep -Seconds 5;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSCopy_localhost_LSTest;
Start-Sleep -Seconds 5;
Start-DbaAgentJob -SqlInstance "localhost\tony" -Job LSRestore_localhost_LSTest;

Name                   
----                   
LSBackup_LSTest        
LSCopy_localhost_LSTest


LSRestore_localhost_...




## Select from Primary and Secondary Tables

In [112]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@
$primary = "localhost";
$secondary = "localhost\tony"
$primarydb = "LSTest";
$secondarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;
Invoke-Sqlcmd -ServerInstance $secondary -Database $secondarydb -Query $select;



Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J         1 Log  Shipping 
FD-PF33VX6J         2 Is   Resilient
FD-PF33VX6J\TONY    1 Log  Shipping 
FD-PF33VX6J\TONY    2 Is   Resilient




## Stop Primary SQL Server Service and Recover Secondary Database
This step simulates a failure of the primary SQL instance and recovers the database on the secondary.

In [93]:
Stop-Service -Name MSSQLSERVER -Force;
Get-Service -Name MSSQLSERVER | Select DisplayName, Status;
Get-Service -Name SQLSERVERAGENT | Select DisplayName, Status;

Invoke-DbaDbLogShipRecovery -SqlInstance 'localhost\tony' -Force;

Stop-Service : Service 'SQL Server (MSSQLSERVER) (MSSQLSERVER)' cannot be stopped due to the following error: Cannot 
open MSSQLSERVER service on computer '.'.
At line:2 char:1
+ Stop-Service -Name MSSQLSERVER -Force;
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : CloseError: (System.ServiceProcess.ServiceController:ServiceController) [Stop-Service],  
   ServiceCommandException
    + FullyQualifiedErrorId : CouldNotStopService,Microsoft.PowerShell.Commands.StopServiceCommand
 
Stop-Service : Collection was modified; enumeration operation may not execute.
At line:2 char:1
+ Stop-Service -Name MSSQLSERVER -Force;
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : NotSpecified: (:) [Stop-Service], InvalidOperationException
    + FullyQualifiedErrorId : System.InvalidOperationException,Microsoft.PowerShell.Commands.StopServiceCommand
 



log shipping.
log shipping.
 shipping.


shipping.
database for log shipping.
log shipping.


Exception calling "Query" with "1" argument(s): "Exception calling "ExecuteWithResults" with "1" argument(s): "Execute 
with results failed for Database 'master'. "" | Error connecting to [localhost\tony]: Exclusive access could not be 
obtained because the database is in use.
RESTORE DATABASE is terminating abnormally.
DisplayName                     Status
-----------                     ------
SQL Server (MSSQLSERVER)       Running
SQL Server Agent (MSSQLSERVER) Running
                                      




## Select from New Primary Table

In [100]:
$select = @"
SELECT 
@@SERVERNAME, * 
FROM TestTable
"@

$primary = "localhost\tony"
$primarydb = "LSTestDR";

Invoke-Sqlcmd -ServerInstance $primary -Database $primarydb -Query $select;


Column1          Col1 Col2 Col3     
-------          ---- ---- ----     
FD-PF33VX6J\TONY    1 Log  Shipping 
FD-PF33VX6J\TONY    2 Is   Resilient


